In [1]:
using CSV
nba = CSV.read("nba.csv", header = 1, types=Dict(:Salary => Int));
position = nba[!, [:C, :PF, :SF, :SG, :PG]];
Salary = nba[:,:Salary];
Score = nba[!,:FantasyScore];

In [2]:
using JuMP, Clp
m = Model()
@variable(m, 1 >= selected[1:444] >= 0)
@constraint(m, sum(selected[i] for i in 1:444) == 12)  #pick up 12 players
@constraint(m, 109000000 .<= sum(selected[i] * Salary[i] for i in 1:444) .<= 132000000)    # Salary Constraint  
@constraint(m, sum(selected[i] * (position[i,:SG]+ position[i,:PG]) for i in 1:444) .>= 6) # more than 6 guards in a team  
@constraint(m, sum(selected[i] * (position[i,:SF]+ position[i,:PF]) for i in 1:444) .>= 4) # more than 4 fronts
@constraint(m, sum(selected[i] * (position[i,:C]) for i in 1:444) .>= 1)  # at least one center
@objective(m, Max, sum(selected[i]*Score[i] for i in 1:444))
;

In [3]:
set_optimizer(m, with_optimizer(Clp.Optimizer))
optimize!(m)
println("termination status: ", termination_status(m))

termination status: OPTIMAL
Coin0506I Presolve 5 (0) rows, 443 (-1) columns and 1487 (-3) elements
Clp0006I 0  Obj 0 Primal inf 92.236468 (5) Dual inf 508.00743 (151)
Clp0006I 4  Obj 107.93799
Clp0000I Optimal - objective value 107.93799
Clp0032I Optimal objective 107.9379883 - 4 iterations time 0.002


In [4]:
y = JuMP.value.(selected)
label = findall(x->x>0.5, y)  # pick up if the esimated value >0.5
nba[label, :Player]

12-element Array{String,1}:
 "James Harden"   
 "Anthony Davis"  
 "Trae Young"     
 "John Collins?"  
 "Luka Doncic"    
 "Brandon Ingram" 
 "Jayson Tatum"   
 "Deandre Ayton?" 
 "Jonathan Isaac?"
 "Pascal Siakam"  
 "Ben Simmons?"   
 "Bam Adebayo"    

In [5]:
sum(Salary[label, :])

125920633

In [6]:
using JuMP, Clp
m = Model()
@variable(m, 1 >= selected[1:444] >= 0)
@constraint(m, sum(selected[i] for i in 1:444) == 8)  
@constraint(m, 109000000 .<= sum(selected[i] * Salary[i] for i in 1:444) .<= 132000000)  
@constraint(m, sum(selected[i] * (position[i,:SG]+ position[i,:PG]) for i in 1:444) .>= 3)  
@constraint(m, sum(selected[i] * (position[i,:SF]+ position[i,:PF]) for i in 1:444) .>= 2)  
@constraint(m, sum(selected[i] * (position[i,:C]) for i in 1:444) .>= 1)  
@objective(m, Max, sum(selected[i]*Score[i] for i in 1:444))
;

In [7]:
set_optimizer(m, with_optimizer(Clp.Optimizer))
optimize!(m)
println("termination status: ", termination_status(m))

termination status: OPTIMAL
Coin0506I Presolve 5 (0) rows, 443 (-1) columns and 1487 (-3) elements
Clp0006I 0  Obj 0 Primal inf 83.932091 (5) Dual inf 508.00743 (151)
Clp0006I 3  Obj 86.531453
Clp0000I Optimal - objective value 86.531453
Clp0032I Optimal objective 86.53145305 - 3 iterations time 0.002


In [8]:
y = JuMP.value.(selected)
label = findall(x->x>0.5, y)
nba[label, :Player]

8-element Array{String,1}:
 "James Harden"       
 "Anthony Davis"      
 "Karl-Anthony Towns?"
 "Trae Young"         
 "John Collins?"      
 "Luka Doncic"        
 "Brandon Ingram"     
 "Pascal Siakam"      